## **Autor: Frogmaker**

In [1]:
# import sys
# !"{sys.executable}" -m pip install beautifulsoup4 --user
from bs4 import BeautifulSoup
import requests

In [2]:
#żródło
baseURL = "http://aligulac.com"
earningsURL = "/earnings/?page="
earningsURLend = "&year=all&country=all&currency=all"

In [3]:
listOfPlayers = []

In [4]:
#pobranie listy zawodników ze źródła


for i in list(range(1, 21)): #82-all, 21- >~1000USD
    response = requests.get(baseURL+earningsURL+str(i)+earningsURLend)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    tags = soup.select('td.ea_name > a')
    
    for tag in tags:
        player = [tag.get_text().strip(), baseURL+tag.get("href")]
        #print(tag.get_text().strip(), ' - ', tag.get("href"))
        listOfPlayers.append(player)

In [5]:
#uzupełnienie danych 
# UWAGA! proces może trwać około 20 minut!
import copy

#player_zero = copy.deepcopy(listOfPlayers[0])
for player in listOfPlayers:
#if True:
    #player = listOfPlayers[0][1]
    response = requests.get(player[1])
    soup = BeautifulSoup(response.content, 'html.parser')
    
    
    race = soup.find('td', string='Race').find_next('td').text.strip()
    country = soup.find('td', string='Country').find_next('td').text.strip()
    if  soup.find('td', string='Full name') is not None:
        name = soup.find('td', string='Full name').find_next('td').text.strip()
    else:
        name = "-"
    if soup.find('td', string='Birthday') is not None:
        birthday = int(soup.find('td', string='Birthday').find_next('td').text.strip()[-4:])
    else:
        birthday = 0
    if soup.find('td', string='Team') is not None:
        team = soup.find('td', string='Team').find_next('td').text.strip()
    else:
        team = "-"
    total_earnings = int(soup.find('td', string='Total earnings').find_next('td').text.strip()[1:].replace(",",""))
    
    for data in [race, country, name, birthday, team, total_earnings]:
        player.append(data)
#print(race, country, name, birthday, team, total_earnings)    
#print(player_zero)    


In [7]:
#inicjacja bazy danych - wykanać raz!

#import sys
#!"{sys.executable}" -m pip install sqlite3 --user

import sqlite3

#nick, url, race, country, name, birthday, team, total_earnings

db = sqlite3.connect('starcraft.db')
cursor = db.cursor()

sqlQuery = "CREATE Table Players (nick TEXT, race TEXT, country TEXT, name NTEXT, birthday INT, team TEXT, totalEarnings INT)"
cursor.execute(sqlQuery)
db.commit()

In [8]:
#funkcja uzupełniająca bazę danych
def insertData(nick, race, country, name, birthday, team, totalEarnings):
    sqlQuery = "INSERT INTO Players (nick, race, country, name, birthday, team, totalEarnings) VALUES ('{}', '{}', '{}', '{}', '{}','{}','{}')"
    sqlQuery = sqlQuery.format(nick, race, country, name, birthday, team, totalEarnings)
    cursor.execute(sqlQuery)
    db.commit()
    pass

# ??? przy próbie wprowadzania 8 kolumn, zamiast 7, nie działa - sprawdzić dlaczego

In [10]:
db = sqlite3.connect('starcraft.db')
cursor = db.cursor()

In [11]:
#uzupełnianie bazy danych:

for player in listOfPlayers:
    insertData(player[0], player[2], player[3], player[4], player[5], player[6], player[7])
db.close()